In [1]:
!pip install timm
!pip install peft
!pip install einops
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
import torchvision
from torch import nn, optim
import timm
import imageio.v2 as imageio
from torchvision.transforms import v2
from glob import glob
from sklearn.preprocessing import LabelEncoder
import time
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torch import linalg as LA
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 727.3 kB/s eta 0:00:00a 0:00:01


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
metadata = pd.read_csv('/kaggle/input/ham1000/dataverse_files — копия/HAM10000_metadata.csv')
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [4]:
first_part = glob('/kaggle/input/ham1000/dataverse_files — копия/HAM10000_images_part_1/*')
second_part = glob('/kaggle/input/ham1000/dataverse_files — копия/HAM10000_images_part_2/*')
first_part.extend(second_part)
first_part_id = [x.split('/')[-1].split('.')[0] for x in first_part]

df_paths = pd.DataFrame({'id': first_part_id, 'path': first_part})
df_paths.head()

,id,path
0,ISIC_0028933,/kaggle/input/ham1000/dataverse_files — копия/...
1,ISIC_0028394,/kaggle/input/ham1000/dataverse_files — копия/...
2,ISIC_0027799,/kaggle/input/ham1000/dataverse_files — копия/...
3,ISIC_0028100,/kaggle/input/ham1000/dataverse_files — копия/...
4,ISIC_0027960,/kaggle/input/ham1000/dataverse_files — копия/...


In [5]:
metadata = metadata.merge(df_paths, left_on='image_id', right_on='id').drop(columns=['id'])

In [6]:
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe.values
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = np.array(imageio.imread(img_path, pilmode='RGB'))
        img = torchvision.transforms.ToTensor()(img)
        #img = Image.open(img_path)
        img = self.transform(img)
        return img, label

In [7]:
transforms_train = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(224, 224)),
    v2.RandomRotation(15),
    v2.RandomHorizontalFlip(p=0.5),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

transforms_test = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(224, 224)),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [8]:
le = LabelEncoder()
metadata['dx_encoded'] = le.fit_transform(metadata['dx'])

In [9]:
train_idx, valid_idx = train_test_split(np.arange(metadata.shape[0]), test_size=0.3,
                                            random_state=0)
trainset = ClassificationDataset(metadata.iloc[train_idx][['path', 'dx_encoded']],
                                 transform=transforms_train)
valset = ClassificationDataset(metadata.iloc[valid_idx][['path', 'dx_encoded']],
                               transform=transforms_test)

def collate_fn(data):
    images, labels = zip(*data)
    images = torch.stack(images)
    labels = torch.tensor(labels)

    return images.float(), labels.long()

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                           shuffle=True, num_workers=2, collate_fn = collate_fn)
val_loader = torch.utils.data.DataLoader(valset, batch_size=32,
                                         shuffle=False, num_workers=2, collate_fn = collate_fn)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## ResNet

In [11]:
model = timm.create_model('resnet50', pretrained=True)
num_classes = 7

num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_features, num_classes)
)
model=model.to(device)

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [12]:
def test(model, loader):
    loss_log = []
    acc_log = []
    model.eval()

    for data, target in loader:
        data = data.to(device)
        target = target.to(device)

        with torch.no_grad():
            predictions = model(data)
            loss = nn.functional.cross_entropy(predictions, target)

            loss_log.append(loss.item())

            acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                   target.cpu()) / target.cpu().shape[0]

            acc_log.append(acc.item())

    return np.mean(loss_log), np.mean(acc_log)

def train_epoch(model, optimizer, train_loader):
    loss_log = []
    acc_log = []
    model.train()

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        predictions = model(data)
        loss = nn.functional.cross_entropy(predictions, target)
        loss.backward()
        optimizer.step()

        loss_log.append(loss.item())

        with torch.no_grad():
            acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                   target.cpu()) / target.cpu().shape[0]

        acc_log.append(acc.item())

    return loss_log, acc_log

def train(model, optimizer, n_epochs, train_loader, val_loader, scheduler=None):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []

    for epoch in range(n_epochs):
        train_loss, train_acc = train_epoch(model, optimizer, train_loader)
        val_loss, val_acc = test(model, val_loader)

        train_loss_log.extend(train_loss)
        train_acc_log.extend(train_acc)

        val_loss_log.append(val_loss)
        val_acc_log.append(val_acc)

        print(f"Epoch {epoch}")
        print(f" train loss: {np.mean(train_loss)}, train acc: {np.mean(train_acc)}")
        print(f" val loss: {val_loss}, val acc: {val_acc}\n")

        if scheduler is not None:
            scheduler.step(val_acc)

    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [13]:
optimizer = optim.Adam(model.parameters())
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, optimizer,
                                                                 10, train_loader, val_loader)

Epoch 0
 train loss: 0.7865359099073843, train acc: 0.7244318181818182
 val loss: 0.6595055194611245, val acc: 0.7736954327593458

Epoch 1
 train loss: 0.5698627220636064, train acc: 0.79375
 val loss: 0.6670140148477351, val acc: 0.7720331987167927

Epoch 2
 train loss: 0.49801584807309235, train acc: 0.8188920454545454
 val loss: 0.5821532759260624, val acc: 0.7914526779600914

Epoch 3
 train loss: 0.4089071396399628, train acc: 0.8509943181818181
 val loss: 0.639537106486077, val acc: 0.7947771460451978

Epoch 4
 train loss: 0.3513155439021913, train acc: 0.8731534090909091
 val loss: 0.49643905968108076, val acc: 0.8280218268962617

Epoch 5
 train loss: 0.31235405636781993, train acc: 0.8849431818181818
 val loss: 0.4509113091737666, val acc: 0.8376627843430702

Epoch 6
 train loss: 0.28685720872811293, train acc: 0.8931818181818182
 val loss: 0.5369006913710148, val acc: 0.821304108868254

Epoch 7
 train loss: 0.23501634642651134, train acc: 0.9105113636363636
 val loss: 0.6593333

In [16]:
import timeit
preds = []
gt = []

loss_log = []
acc_log = []
model.eval()
    
startT = timeit.default_timer() 
for data, target in val_loader:
    data = data.to(device)
    target = target.to(device)

    with torch.no_grad():
        predictions = model(data)
        loss = nn.functional.cross_entropy(predictions, target)

        loss_log.append(loss.item())

        acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                target.cpu()) / target.cpu().shape[0]

        acc_log.append(acc.item())
        
        for i in range(len(target)):
            preds.append(torch.argmax(predictions[i].cpu()))
            gt.append(target[i].cpu())
            
endT = timeit.default_timer()
run_time = endT-startT

In [17]:
from sklearn.metrics import classification_report
print(classification_report(np.squeeze(gt), np.squeeze(preds)))

              precision    recall  f1-score   support

           0       0.58      0.52      0.55        93
           1       0.86      0.29      0.44       164
           2       0.39      0.93      0.54       315
           3       0.95      0.41      0.57        44
           4       0.62      0.36      0.46       338
           5       0.94      0.87      0.90      2012
           6       0.85      0.90      0.88        39

    accuracy                           0.77      3005
   macro avg       0.74      0.61      0.62      3005
weighted avg       0.83      0.77      0.77      3005



## ViT base

In [18]:
model = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
num_classes = 7

num_features = model.head.in_features

model.head = nn.Sequential(
    nn.Linear(num_features, num_classes)
)
model=model.to(device)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [19]:
optimizer = optim.Adam(model.parameters())
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, optimizer,
                                                                 10, train_loader, val_loader)

Epoch 0
 train loss: 1.2448290841145948, train acc: 0.6475852272727273
 val loss: 1.028932271802679, val acc: 0.6675761189866574

Epoch 1
 train loss: 0.9554861247034172, train acc: 0.665625
 val loss: 1.0812673841385132, val acc: 0.6695364085917778

Epoch 2
 train loss: 0.9119417255575006, train acc: 0.6764204545454545
 val loss: 1.049151655841381, val acc: 0.6459326851875224

Epoch 3
 train loss: 0.8845450619404966, train acc: 0.6818181818181818
 val loss: 0.9836938825059445, val acc: 0.6579007702939054

Epoch 4
 train loss: 0.8490247857841579, train acc: 0.6931818181818182
 val loss: 0.969762884555979, val acc: 0.6729640497806224

Epoch 5
 train loss: 0.8363007584756071, train acc: 0.6936079545454545
 val loss: 0.8424312795730348, val acc: 0.6819401136104096

Epoch 6
 train loss: 0.8334008894183419, train acc: 0.6927556818181818
 val loss: 0.8861818047280007, val acc: 0.6792461478963812

Epoch 7
 train loss: 0.8423483331095089, train acc: 0.6896306818181818
 val loss: 0.958151446377

In [20]:
preds = []
gt = []

loss_log = []
acc_log = []
model.eval()
    
startT = timeit.default_timer() 
for data, target in val_loader:
    data = data.to(device)
    target = target.to(device)

    with torch.no_grad():
        predictions = model(data)
        loss = nn.functional.cross_entropy(predictions, target)

        loss_log.append(loss.item())

        acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                target.cpu()) / target.cpu().shape[0]

        acc_log.append(acc.item())
        
        for i in range(len(target)):
            preds.append(torch.argmax(predictions[i].cpu()))
            gt.append(target[i].cpu())
            
endT = timeit.default_timer()
run_time = endT-startT

In [21]:
print(classification_report(np.squeeze(gt), np.squeeze(preds)))

              precision    recall  f1-score   support

           0       0.19      0.10      0.13        93
           1       0.50      0.03      0.06       164
           2       0.34      0.05      0.09       315
           3       0.00      0.00      0.00        44
           4       0.24      0.27      0.25       338
           5       0.74      0.92      0.82      2012
           6       0.40      0.26      0.31        39

    accuracy                           0.66      3005
   macro avg       0.35      0.23      0.24      3005
weighted avg       0.60      0.66      0.60      3005



## Base dinov2

In [22]:
model = timm.create_model('vit_base_patch14_dinov2.lvd142m', pretrained=True)
model.eval()
model = model.to(device)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [46]:
trainset = ClassificationDataset(metadata.iloc[train_idx][['path', 'dx_encoded']],
                                 transform=transforms_train)
valset = ClassificationDataset(metadata.iloc[valid_idx][['path', 'dx_encoded']],
                               transform=transforms_test)

transforms_train = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(518, 518)),
    v2.RandomRotation(15),
    v2.RandomHorizontalFlip(p=0.5),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

transforms_test = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(518, 518)),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                           shuffle=True, num_workers=2, collate_fn = collate_fn)
val_loader = torch.utils.data.DataLoader(valset, batch_size=32,
                                         shuffle=False, num_workers=2, collate_fn = collate_fn)

In [27]:
from tqdm.notebook import tqdm
embeddings = []
labels = []
for i in tqdm(range(len(trainset))):
    data, label = trainset[i]
    embeddings.append(model(data.unsqueeze(0).to(device)).cpu().detach().numpy())
    labels.append(label)

  0%|          | 0/7010 [00:00<?, ?it/s]

In [35]:
np.squeeze(np.array(embeddings), 1).shape

(7010, 768)

In [36]:
from sklearn import svm

clf = svm.SVC(gamma='scale')

clf.fit(np.squeeze(np.array(embeddings), 1), labels)

SVC()

In [37]:
predictions = clf.predict(np.squeeze(np.array(embeddings), 1))

In [38]:
sum(predictions==np.array(labels))/predictions.shape[0]

0.8492154065620542

In [39]:
embeddings_test = []
labels_test = []
for i in tqdm(range(len(valset))):
    data, label = valset[i]
    embeddings_test.append(model(data.unsqueeze(0).to(device)).cpu().detach().numpy())
    labels_test.append(label)

  0%|          | 0/3005 [00:00<?, ?it/s]

In [40]:
predictions_test = clf.predict(np.squeeze(np.array(embeddings_test), 1))

In [41]:
sum(predictions_test==np.array(labels_test))/predictions_test.shape[0]

0.8076539101497504

In [42]:
print(classification_report(np.array(labels_test), predictions_test))

              precision    recall  f1-score   support

           0       0.58      0.15      0.24        93
           1       0.72      0.68      0.70       164
           2       0.59      0.68      0.63       315
           3       1.00      0.05      0.09        44
           4       0.58      0.37      0.45       338
           5       0.87      0.96      0.92      2012
           6       0.95      0.46      0.62        39

    accuracy                           0.81      3005
   macro avg       0.76      0.48      0.52      3005
weighted avg       0.80      0.81      0.79      3005



## Large dinov2

In [43]:
model = timm.create_model('vit_large_patch14_dinov2.lvd142m', pretrained=True)
model.eval()
model = model.to(device)

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [44]:
data_config = timm.data.resolve_model_data_config(model)
transforms_dino = timm.data.create_transform(**data_config, is_training=False)

In [47]:
from tqdm.notebook import tqdm
embeddings = []
labels = []

#trainset_dino = ClassificationDataset(metadata.iloc[train_idx][['path', 'dx_encoded']],
#                                 transform=transforms_dino)
#train_loader_dino = torch.utils.data.DataLoader(trainset_dino, batch_size=32,shuffle=False,
#                                           collate_fn = collate_fn, num_workers=2)

for data, target in tqdm(train_loader):
    data = data.to(device)

    with torch.no_grad():
        embeddings.extend(model(data).cpu().numpy())
        
    labels.extend(target)
    
    

  0%|          | 0/220 [00:00<?, ?it/s]

In [48]:
from sklearn import svm

clf = svm.SVC(gamma='scale')

clf.fit(np.array(embeddings), labels)

SVC()

In [49]:
predictions = clf.predict(np.array(embeddings))
sum(predictions==np.array(labels))/predictions.shape[0]

0.8584878744650499

In [51]:
embeddings_val = []
labels_val = []

for data, target in tqdm(val_loader):
    data = data.to(device)

    with torch.no_grad():
        embeddings_val.extend(model(data).cpu().numpy())
        
    labels_val.extend(target)

  0%|          | 0/94 [00:00<?, ?it/s]

In [53]:
predictions = np.squeeze(clf.predict(np.array(embeddings_val)))
sum(predictions==np.array(labels_val))/predictions.shape[0]

0.8149750415973378

In [54]:
print(classification_report(np.array(labels_val), predictions))

              precision    recall  f1-score   support

           0       0.61      0.30      0.40        93
           1       0.72      0.70      0.71       164
           2       0.60      0.69      0.64       315
           3       1.00      0.02      0.04        44
           4       0.64      0.39      0.49       338
           5       0.88      0.96      0.92      2012
           6       0.96      0.59      0.73        39

    accuracy                           0.81      3005
   macro avg       0.77      0.52      0.56      3005
weighted avg       0.81      0.81      0.80      3005



## Swinv2

In [55]:
model = timm.create_model('swinv2_base_window8_256.ms_in1k', pretrained=True)
model.eval()
model = model.to(device)

model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

In [56]:
transforms_256 = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(256, 256)),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
trainset_256 = ClassificationDataset(metadata.iloc[train_idx][['path', 'dx_encoded']],
                                 transform=transforms_256)
valset_256 = ClassificationDataset(metadata.iloc[valid_idx][['path', 'dx_encoded']],
                                 transform=transforms_256)
train_loader_256 = torch.utils.data.DataLoader(trainset_256, batch_size=64,
                                                      shuffle=False,
                                           collate_fn = collate_fn)
val_loader_256 = torch.utils.data.DataLoader(valset_256, batch_size=64,
                                         shuffle=False, collate_fn = collate_fn)

In [57]:
from tqdm.notebook import tqdm
embeddings = []
labels = []

for data, target in tqdm(train_loader_256):
    data = data.to(device)

    with torch.no_grad():
        embeddings.extend(model(data).cpu().numpy())
        
    labels.extend(target)
    

  0%|          | 0/110 [00:00<?, ?it/s]

In [58]:
from sklearn import svm

clf = svm.SVC(gamma='scale')

clf.fit(np.array(embeddings), labels)

SVC()

In [59]:
predictions = clf.predict(np.array(embeddings))
sum(predictions==np.array(labels))/predictions.shape[0]

0.8114122681883025

In [60]:
embeddings_val = []
labels_val = []

for data, target in tqdm(val_loader_256):
    data = data.to(device)

    with torch.no_grad():
        embeddings_val.extend(model(data).cpu().numpy())
        
    labels_val.extend(target)

  0%|          | 0/47 [00:00<?, ?it/s]

In [61]:
predictions = np.squeeze(clf.predict(np.array(embeddings_val)))
sum(predictions==np.array(labels_val))/predictions.shape[0]

0.762063227953411

In [62]:
print(classification_report(np.array(labels_val), predictions))

              precision    recall  f1-score   support

           0       0.59      0.29      0.39        93
           1       0.63      0.35      0.45       164
           2       0.55      0.50      0.52       315
           3       1.00      0.02      0.04        44
           4       0.66      0.23      0.34       338
           5       0.80      0.97      0.88      2012
           6       1.00      0.21      0.34        39

    accuracy                           0.76      3005
   macro avg       0.75      0.37      0.42      3005
weighted avg       0.75      0.76      0.72      3005



## Base dinov2 + linear layer and peft training

In [63]:
model = timm.create_model('vit_base_patch14_dinov2.lvd142m', pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
num_classes = 7

num_features = 768

model.head = nn.Sequential(
    nn.Linear(num_features, num_classes)
)
model=model.to(device)    

for name, param in model.named_parameters():
    print(name,param.requires_grad)

cls_token False
pos_embed False
patch_embed.proj.weight False
patch_embed.proj.bias False
blocks.0.norm1.weight False
blocks.0.norm1.bias False
blocks.0.attn.qkv.weight False
blocks.0.attn.qkv.bias False
blocks.0.attn.proj.weight False
blocks.0.attn.proj.bias False
blocks.0.ls1.gamma False
blocks.0.norm2.weight False
blocks.0.norm2.bias False
blocks.0.mlp.fc1.weight False
blocks.0.mlp.fc1.bias False
blocks.0.mlp.fc2.weight False
blocks.0.mlp.fc2.bias False
blocks.0.ls2.gamma False
blocks.1.norm1.weight False
blocks.1.norm1.bias False
blocks.1.attn.qkv.weight False
blocks.1.attn.qkv.bias False
blocks.1.attn.proj.weight False
blocks.1.attn.proj.bias False
blocks.1.ls1.gamma False
blocks.1.norm2.weight False
blocks.1.norm2.bias False
blocks.1.mlp.fc1.weight False
blocks.1.mlp.fc1.bias False
blocks.1.mlp.fc2.weight False
blocks.1.mlp.fc2.bias False
blocks.1.ls2.gamma False
blocks.2.norm1.weight False
blocks.2.norm1.bias False
blocks.2.attn.qkv.weight False
blocks.2.attn.qkv.bias False
bloc

In [64]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["head.0"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
model = get_peft_model(model, config)
model=model.to(device)
model.print_trainable_parameters()

trainable params: 12,400 || all params: 86,597,495 || trainable%: 0.0143


In [65]:
optimizer = optim.Adam(model.parameters())
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, optimizer,
                                                                 10, train_loader, val_loader)

Epoch 0
 train loss: 1.0202400043606759, train acc: 0.6488636363636363
 val loss: 0.8231396345382042, val acc: 0.7016576484162756

Epoch 1
 train loss: 0.7373947593298825, train acc: 0.7289772727272728
 val loss: 0.7446067761233512, val acc: 0.7342030450384668

Epoch 2
 train loss: 0.6250645535913381, train acc: 0.7713068181818182
 val loss: 0.636304590296238, val acc: 0.7634927547992544

Epoch 3
 train loss: 0.5807541770013895, train acc: 0.7869318181818182
 val loss: 0.642707052065971, val acc: 0.7614636833363391

Epoch 4
 train loss: 0.5488276739012111, train acc: 0.7988636363636363
 val loss: 0.5951205650542645, val acc: 0.7890911590545735

Epoch 5
 train loss: 0.5355052097277208, train acc: 0.8039772727272727
 val loss: 0.5723114796775453, val acc: 0.7917507335226587

Epoch 6
 train loss: 0.5165385353971611, train acc: 0.8085227272727272
 val loss: 0.5616588145494461, val acc: 0.7963705982299562

Epoch 7
 train loss: 0.4954388031228022, train acc: 0.8167613636363636
 val loss: 0.5

In [66]:
preds = []
gt = []

loss_log = []
acc_log = []
model.eval()
    
startT = timeit.default_timer() 
for data, target in val_loader:
    data = data.to(device)
    target = target.to(device)

    with torch.no_grad():
        predictions = model(data)
        loss = nn.functional.cross_entropy(predictions, target)

        loss_log.append(loss.item())

        acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                target.cpu()) / target.cpu().shape[0]

        acc_log.append(acc.item())
        
        for i in range(len(target)):
            preds.append(torch.argmax(predictions[i].cpu()))
            gt.append(target[i].cpu())
            
endT = timeit.default_timer()
run_time = endT-startT

In [67]:
print(classification_report(gt, preds))

              precision    recall  f1-score   support

           0       0.48      0.12      0.19        93
           1       0.76      0.65      0.70       164
           2       0.49      0.73      0.59       315
           3       0.65      0.30      0.41        44
           4       0.59      0.36      0.44       338
           5       0.89      0.94      0.91      2012
           6       0.90      0.69      0.78        39

    accuracy                           0.80      3005
   macro avg       0.68      0.54      0.58      3005
weighted avg       0.79      0.80      0.78      3005

